# 1 Scraping Birth Control App Data from the Apple App Store

- This notebook uses the app_store_scraper package to scrape the Apple app store for the reviews of 4 birth control apps to gain insight into women's current issues two years after the overturn of Roe vs. Wade.

- I uses the sklearn, pystemmer and eli5 packages to build a classifer to predict whether users who wrote 1-star and 2-star reviews on 4 apps expressed that they were [+ categories]

- Apps were selected based on search results for "birth control" on the app store, as well as online reviews of mental health apps. The dataframe containing their names and ids is manually created here.

## 1. Install packages to get prepared

In [1]:
#!pip install itunes-app-scraper-dmi==0.9.4
#!pip install app-store-scraper==0.3.5

In [2]:
import pandas as pd

from itunes_app_scraper.scraper import AppStoreScraper
from app_store_scraper import AppStore

import time
import re

import requests
from plotnine import * 

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", 300)

## 2. Building dataframe of the 7 apps

This section builds a dataframe of the birth control apps to analyze from the App Store website (https://www.apple.com/app-store/). The names and ids of the apps are in each of their App store URLs

The apps are:
- APP 01: [Nurx: Birth Control Delivered](https://apps.apple.com/us/app/nurx-birth-control-delivered/id1213141301) 
- APP 02: [Natural Cycles: Birth Control](https://apps.apple.com/us/app/natural-cycles-birth-control/id765535549)
- APP 03: [myPill Birth Control Reminder](https://apps.apple.com/us/app/mypill-birth-control-reminder/id425632209)
- APP 04: [Birth Control Pill Reminder](https://apps.apple.com/us/app/birth-control-pill-reminder/id953740058) 
- APP 05: [Planned Parenthood Direct](https://apps.apple.com/us/app/planned-parenthood-direct/id1214393415)
- APP 06: [Flo Period & Pregnancy Tracker](https://apps.apple.com/us/app/flo-period-pregnancy-tracker/id1038369065)
- APP 07: [Clue Period Tracker & Calendar](https://apps.apple.com/us/app/clue-period-tracker-calendar/id657189652)


In [8]:
app_names_ids = [
    {'name':'nurx-birth-control-delivered',
    'id': 1213141301},
    {'name':'natural-cycles-birth-control',
    'id':765535549},
    {'name':'mypill-birth-control-reminder',
    'id': 425632209},
    {'name': 'birth-control-pill-reminder',
    'id':953740058},
    {'name': 'planned-parenthood-direct',
    'id':1214393415},
    {'name':'flo-period-pregnancy-tracker',
    'id': 1038369065},
    {'name':'clue-period-tracker-calendar',
    'id': 657189652},
]

In [9]:
birthControl_apps = pd.DataFrame(app_names_ids)
birthControl_apps

,name,id
0,nurx-birth-control-delivered,1213141301
1,natural-cycles-birth-control,765535549
2,mypill-birth-control-reminder,425632209
3,birth-control-pill-reminder,953740058
4,planned-parenthood-direct,1214393415
5,flo-period-pregnancy-tracker,1038369065
6,clue-period-tracker-calendar,657189652


## 3. Scraping the reviews of 7 apps
- timeline: from 2020-05-02 to 2024-05-02 (4 years)
- number: all the reviews within the described timeline above

In [10]:
import datetime

all_7apps_reviews = []

for app in app_names_ids:
    my_app = AppStore(
      country='us',       
      app_name=app['name'], 
      app_id=app['id']    
    ) 

    my_app.review(
      after=pd.to_datetime('2020-05-02'),
      sleep=1,
    )
    
    reviews = my_app.reviews
    
    # Filter reviews to include only those before the end date
    end_date = datetime.datetime(2024, 5, 3)
    filtered_reviews = [review for review in reviews if review['date'] < end_date]
    
    for review in filtered_reviews:
        review['app_name'] = app['name']
        review['app_id'] = app['id']
        
    reviews_df = pd.DataFrame(filtered_reviews)
    
    all_7apps_reviews.append(reviews_df)
    time.sleep(5)


2024-05-05 21:28:32,624 [INFO] Base - Initialised: AppStore('us', 'nurx-birth-control-delivered', 1213141301)
2024-05-05 21:28:32,625 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/nurx-birth-control-delivered/id1213141301
2024-05-05 21:28:38,418 [INFO] Base - [id:1213141301] Fetched 67 reviews (67 fetched in total)
2024-05-05 21:28:45,393 [INFO] Base - [id:1213141301] Fetched 140 reviews (140 fetched in total)
2024-05-05 21:28:52,421 [INFO] Base - [id:1213141301] Fetched 223 reviews (223 fetched in total)
2024-05-05 21:28:59,349 [INFO] Base - [id:1213141301] Fetched 309 reviews (309 fetched in total)
2024-05-05 21:29:06,403 [INFO] Base - [id:1213141301] Fetched 388 reviews (388 fetched in total)
2024-05-05 21:29:13,315 [INFO] Base - [id:1213141301] Fetched 481 reviews (481 fetched in total)
2024-05-05 21:29:20,592 [INFO] Base - [id:1213141301] Fetched 569 reviews (569 fetched in total)
2024-05-05 21:29:27,660 [INFO] Base - [id:1213141301] Fetched 663 reviews 

2024-05-05 21:36:54,662 [INFO] Base - [id:1214393415] Fetched 258 reviews (258 fetched in total)
2024-05-05 21:36:59,396 [INFO] Base - [id:1214393415] Fetched 293 reviews (293 fetched in total)
2024-05-05 21:37:04,777 [INFO] Base - Initialised: AppStore('us', 'flo-period-pregnancy-tracker', 1038369065)
2024-05-05 21:37:04,778 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/flo-period-pregnancy-tracker/id1038369065
2024-05-05 21:37:09,937 [INFO] Base - [id:1038369065] Fetched 65 reviews (65 fetched in total)
2024-05-05 21:37:16,961 [INFO] Base - [id:1038369065] Fetched 146 reviews (146 fetched in total)
2024-05-05 21:37:23,472 [INFO] Base - [id:1038369065] Fetched 221 reviews (221 fetched in total)
2024-05-05 21:37:29,778 [INFO] Base - [id:1038369065] Fetched 293 reviews (293 fetched in total)
2024-05-05 21:37:36,161 [INFO] Base - [id:1038369065] Fetched 366 reviews (366 fetched in total)
2024-05-05 21:37:42,928 [INFO] Base - [id:1038369065] Fetched 440 reviews 

2024-05-05 21:46:04,019 [INFO] Base - [id:1038369065] Fetched 5525 reviews (5525 fetched in total)
2024-05-05 21:46:10,980 [INFO] Base - [id:1038369065] Fetched 5592 reviews (5592 fetched in total)
2024-05-05 21:46:17,575 [INFO] Base - [id:1038369065] Fetched 5653 reviews (5653 fetched in total)
2024-05-05 21:46:24,664 [INFO] Base - [id:1038369065] Fetched 5725 reviews (5725 fetched in total)
2024-05-05 21:46:31,809 [INFO] Base - [id:1038369065] Fetched 5790 reviews (5790 fetched in total)
2024-05-05 21:46:38,392 [INFO] Base - [id:1038369065] Fetched 5859 reviews (5859 fetched in total)
2024-05-05 21:46:45,458 [INFO] Base - [id:1038369065] Fetched 5927 reviews (5927 fetched in total)
2024-05-05 21:46:52,209 [INFO] Base - [id:1038369065] Fetched 5993 reviews (5993 fetched in total)
2024-05-05 21:46:58,958 [INFO] Base - [id:1038369065] Fetched 6059 reviews (6059 fetched in total)
2024-05-05 21:47:05,773 [INFO] Base - [id:1038369065] Fetched 6125 reviews (6125 fetched in total)
2024-05-05

2024-05-05 21:55:30,372 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:55:36,999 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:55:44,172 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:55:50,866 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:55:57,239 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:56:03,742 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:56:10,438 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:56:16,938 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:56:24,493 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05 21:56:30,705 [INFO] Base - [id:1038369065] Fetched 9753 reviews (9753 fetched in total)
2024-05-05

2024-05-05 22:04:55,802 [INFO] Base - [id:1038369065] Fetched 10280 reviews (10280 fetched in total)
2024-05-05 22:05:01,298 [INFO] Base - Initialised: AppStore('us', 'clue-period-tracker-calendar', 657189652)
2024-05-05 22:05:01,299 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/clue-period-tracker-calendar/id657189652
2024-05-05 22:05:07,490 [INFO] Base - [id:657189652] Fetched 66 reviews (66 fetched in total)
2024-05-05 22:05:13,969 [INFO] Base - [id:657189652] Fetched 144 reviews (144 fetched in total)
2024-05-05 22:05:20,530 [INFO] Base - [id:657189652] Fetched 208 reviews (208 fetched in total)
2024-05-05 22:05:27,959 [INFO] Base - [id:657189652] Fetched 269 reviews (269 fetched in total)
2024-05-05 22:05:34,285 [INFO] Base - [id:657189652] Fetched 333 reviews (333 fetched in total)
2024-05-05 22:05:41,207 [INFO] Base - [id:657189652] Fetched 396 reviews (396 fetched in total)
2024-05-05 22:05:47,720 [INFO] Base - [id:657189652] Fetched 460 reviews (460 

2024-05-05 22:14:13,749 [INFO] Base - [id:657189652] Fetched 4038 reviews (4038 fetched in total)
2024-05-05 22:14:20,147 [INFO] Base - [id:657189652] Fetched 4073 reviews (4073 fetched in total)
2024-05-05 22:14:26,502 [INFO] Base - [id:657189652] Fetched 4123 reviews (4123 fetched in total)
2024-05-05 22:14:33,047 [INFO] Base - [id:657189652] Fetched 4160 reviews (4160 fetched in total)
2024-05-05 22:14:39,724 [INFO] Base - [id:657189652] Fetched 4204 reviews (4204 fetched in total)
2024-05-05 22:14:47,086 [INFO] Base - [id:657189652] Fetched 4243 reviews (4243 fetched in total)
2024-05-05 22:14:53,357 [INFO] Base - [id:657189652] Fetched 4289 reviews (4289 fetched in total)
2024-05-05 22:15:00,224 [INFO] Base - [id:657189652] Fetched 4330 reviews (4330 fetched in total)
2024-05-05 22:15:06,694 [INFO] Base - [id:657189652] Fetched 4377 reviews (4377 fetched in total)
2024-05-05 22:15:13,446 [INFO] Base - [id:657189652] Fetched 4412 reviews (4412 fetched in total)
2024-05-05 22:15:21,

2024-05-05 22:23:38,230 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:23:45,870 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:23:52,461 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:23:58,762 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:05,588 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:11,907 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:19,441 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:25,907 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:32,390 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:38,640 [INFO] Base - [id:657189652] Fetched 4619 reviews (4619 fetched in total)
2024-05-05 22:24:46,

In [11]:
all_7apps_reviews_df = pd.concat(all_7apps_reviews)
all_7apps_reviews_df

,date,developerResponse,review,rating,isEdited,userName,title,app_name,app_id
0,2021-07-15 00:23:10,"{'id': 18105323, 'body': 'Hi Kristena, thank you for you amazing review! If you ever have any concerns or additional feedback, please feel free to email reviewsupport@nurx.co. Have a great week!', 'modified': '2020-09-25T18:07:58Z'}","I've seriously never written a review for an app before but I love Nurx!!! I've never had such an easy experience obtaining birth control. Ever. It was so seamless and easy, and the staff communicates with you at every single step. You can literally message with an RN one on one with any questio...",5,True,kristenamaya,Absolutely amazing. Highly recommend to everyone I know!!!,nurx-birth-control-delivered,1213141301
1,2021-07-20 19:30:18,"{'id': 24106695, 'body': 'Hi, thank you for your incredible review, you made our week! We are beyond delighted to hear of your experience. If we can ever be of assistance, please do not hesitate to reach out at reviewsupport@nurx.co as we are always happy to help', 'modified': '2021-07-24T21:20:...","I’ve used Nurx for several years - first for birth control and now for migraines - and always had such great experiences with both the providers and the customer service reps. I especially want to mention Karolann, who I spoke to recently on the migraine CS line: she was just so, so kind and pat...",5,False,gbondy1,Incredible for migraines!,nurx-birth-control-delivered,1213141301
2,2022-06-28 22:13:51,"{'id': 30650880, 'body': 'We are sorry to hear about your experience. Nurx has over 400K patients, many of whom have trusted us to care for additional health needs beyond birth control. We pride ourselves on helping our patients get what they need. Please email reviewsupport@nurx.co to best loca...",This whole business is a scam. It’s an exhausting and consistently frustrating take on a crucial service. I absolutely love the concept of getting it delivered and being able to do it all online. First month was fine. Every month since they have tried charging my card anywhere from $8-$10 even t...,1,False,Emilie Bo-Bemely.,What a Nightmare,nurx-birth-control-delivered,1213141301
3,2023-11-30 22:46:13,NaN,I was happy with my first order of 1 pack of birth control which was covered by my insurance and I had no issues. My second order is where things got really shady. They told me that my insurance wouldn’t cover my prescription so I would have to pay out of pocket. I called my insurance and asked ...,1,False,moldyspice,It’s a scam,nurx-birth-control-delivered,1213141301
4,2020-09-10 17:26:50,NaN,I had been going back and forth getting my prescription from my PEDIATRICIAN (spoil alert- I'm 20). Nurx let me get my birth control without dealing with annoying doctor's offices and trying to get an appointment. I just told them exactly what I wanted (the same birth control I've been on for ye...,5,False,beccaem2472,Helpful app and AMAZING service,nurx-birth-control-delivered,1213141301
...,...,...,...,...,...,...,...,...,...
4790,2020-06-29 01:35:03,"{'id': 16309508, 'body': 'Hello! We want Clue to be a sustainable app thanks to our users: we don't want to show ads, nor sell your data: we want to create a real value in your lives. And the best way to do that was to include some features under Clue Plus. Please reach us from the app, we'll b...","I really like this app! I love being able to track a lot of different things with my cycle as I do have PCOS and sometimes that makes it difficult to keep track of certain things. \nThe only problem that I have been having, and it’s been happening for over a month now is that when I try to see m...",1,False,Kali-Luna,Edited review after company wants paid billing monthly for a once FREE app.,clue-period-tracker-calendar,657189652
4791,2020-05-16 01:31:21,NaN,This all helps.me allot and so I would recommend it!!,4,False,klgs_hhs,Clue is great!!,clue-period-tracker-calendar,657189652
4792,2020-05-12 16:50:28,NaN,Hi so I have a YouTube channe

In [12]:
#Quick Check1: How many reviews does each app have?
all_7apps_reviews_df.groupby('app_name').size().sort_values(ascending=False)

app_name
flo-period-pregnancy-tracker     10274
clue-period-tracker-calendar      4795
nurx-birth-control-delivered      2630
natural-cycles-birth-control       886
planned-parenthood-direct          292
birth-control-pill-reminder         37
mypill-birth-control-reminder       35
dtype: int64

In [13]:
#Quick Check2: How many reviews are there for each rating level?
all_7apps_reviews_df.rating.value_counts()

1    6481
5    6164
2    2344
3    2205
4    1755
Name: rating, dtype: int64

In [15]:
#all_7apps_reviews_df.to_csv('all_7apps_reviews.csv')